# Merge all Hospital Archives dataframes

In [2]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
import csv
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

main_dir = os.path.expanduser("~/GitHub/hospitals-data-archive/")


## Create file of feature categories

In [16]:
ls2 = ['HVBP Total Performance',
       'HVBP Clinical Outcomes',
       'HVBP Efficiency',
       'HVBP Safety',
       'HVBP HCAHPS',
       'Outpatient Imaging Efficiency',
       'Payment and Value of Care',
       'Complications and Deaths',
       'Hospital Readmissions Reduction Program',
       'Hospital Acquired Conditions Reduction Program',
       'HCAHPS',
       'Timely and Effective Care',
       'Unplanned Visits',
]
with open(main_dir + '/files_for_app/categories.csv', 'w+', newline='') as OUT:
    writer = csv.writer(OUT)
    writer.writerow(ls2)


## Load curated/compiled hospital archives data

In [26]:
df = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/hai_df.pkl.gz', compression='gzip')
print('df.shape:', df.shape)
for i in list(df):
    if i not in ['Facility ID', 'Facility Name', 'file_month','file_year']:
        df.rename(columns={i: 'Healthcare Associated Infections: ' + i}, inplace=True)

ls1 = [#'hvbp_total_performance_scores_df.pkl.gz', 
       #'hvbp_clinical_outcomes_df.pkl.gz', 
       #'hvbp_efficiency_df.pkl.gz',
       #'hvbp_safety_df.pkl.gz', 
       #'hvbp_hcahps_df.pkl.gz',
       'Outpatient_Imaging_Efficiency_df.pkl.gz', 
       #'Payment_and_ValueOfCare_df.pkl.gz', 
       #'complication_and_deaths_df.pkl.gz', 
       #'hrrp_df.pkl.gz',
       #'hacrp_df.pkl.gz',
       #'hcahps_df.pkl.gz',
       #'Timely_and_Effective_Care_df.pkl.gz',
       #'unplanned_visits_df.pkl.gz',
      ]

brk = 0
for i, l in enumerate(ls1):
    print(l)
    tdf = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/' + l, compression='gzip')
    
    for ii in list(tdf):
        if ii not in ['Facility ID', 'Facility Name', 'file_month','file_year']:
            tdf.rename(columns={ii: ls2[i] + ': ' + ii}, inplace=True)
    
    df = df.merge(tdf, how='outer')
    print('df.shape:', df.shape)
    
    for iii in list(df):
        if '_x' in iii:
            print('redundant column name:', iii)
            brk = 1
            break
            
    if brk == 1: break
    del tdf

print('df.shape:', df.shape)
df.head()


df.shape: (195832, 44)
Outpatient_Imaging_Efficiency_df.pkl.gz
df.shape: (200003, 51)
df.shape: (200003, 51)


,Facility ID,Facility Name,file_month,file_year,Healthcare Associated Infections: CLABSI (SIR),Healthcare Associated Infections: CLABSI Number of Device Days,Healthcare Associated Infections: CLABSI Number of Procedures,Healthcare Associated Infections: CLABSI Observed Cases,Healthcare Associated Infections: CLABSI Predicted Cases,Healthcare Associated Infections: CLABSI SIR (ICUs only),Healthcare Associated Infections: CLABSI lower CL,Healthcare Associated Infections: CLABSI upper CL,Healthcare Associated Infections: CAUTI (SIR),Healthcare Associated Infections: CAUTI Number of Procedures,Healthcare Associated Infections: CAUTI Observed Cases,Healthcare Associated Infections: CAUTI Predicted Cases,Healthcare Associated Infections: CAUTI SIR (ICUs only),Healthcare Associated Infections: CAUTI Urinary Catheter Days,Healthcare Associated Infections: CAUTI lower CL,Healthcare Associated Infections: CAUTI upper CL,Healthcare Associated Infections: MRSA (SIR),Healthcare Associated Infections: MRSA Observed Cases,Healthcare Associated Infections: MRSA Predicted Cases,Healthcare Associated Infections: MRSA lower CL,Healthcare Associated Infections: MRSA patient days,Healthcare Associated Infections: MRSA upper CL,Healthcare Associated Infections: CDIFF (SIR),Healthcare Associated Infections: CDIFF Observed Cases,Healthcare Associated Infections: CDIFF Predicted Cases,Healthcare Associated Infections: CDIFF lower CL,Healthcare Associated Infections: CDIFF patient days,Healthcare Associated Infections: CDIFF upper CL,Healthcare Associated Infections: SSI Colon Number of Procedures,Healthcare Associated Infections: SSI Colon Observed Cases,Healthcare Associated Infections: SSI Colon Predicted Cases,Healthcare Associated Infections: SSI Colon Surgery (SIR),Healthcare Associated Infections: SSI Colon lower CL,Healthcare Associated Infections: SSI Colon upper CL,Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR),Healthcare Associated Infections: SSI Abdominal Number of Procedures,Healthcare Associated Infections: SSI Abdominal Observed Cases,Healthcare Associated Infections: SSI Abdominal Predicted Cases,Healthcare Associated Infections: SSI Abdominal lower CL,Healthcare Associated Infections: SSI Abdominal upper CL,HVBP Total Performance: Abdomen CT Use of Contrast Material,HVBP Total Performance: Breast Cancer Screening Recall Rates,HVBP Total Performance: MRI Lumbar Spine for Low Back Pain,HVBP Total Performance: Mammography Follow-up Rates,HVBP Total Performance: Outpatients who got cardiac imaging stress tests before low-risk outpatient surgery,HVBP Total Performance: Outpatients with brain CT scans who got a sinus CT scan at the same time,HVBP Total Performance: Thorax CT Use of Contrast Material
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,01,2023,0.661,10024.0,NaN,7.0,10.597,NaN,0.289,1.307,0.300,NaN,8.0,26.630,NaN,17731.0,0.140,0.570,0.850,8.0,9.412,0.395,101908.0,1.614,0.660,48.0,72.686,0.492,101451.0,0.868,154.0,5.0,4.548,1.099,0.403,2.437,0.0,200.0,0.0,1.845,NaN,1.624,5.7,5.5,42.5,NaN,6.8,NaN,NaN
1,010005,MARSHALL MEDICAL CENTERS,01,2023,3.673,3713.0,NaN,9.0,2.450,NaN,1.791,6.741,1.201,NaN,6.0,4.995,NaN,8670.0,0.487,2.498,0.000,0.0,1.999,NaN,38413.0,1.499,0.858,9.0,10.484,0.419,35686.0,1.575,88.0,3.0,2.512,1.194,0.304,3.250,NaN,38.0,0.0,0.316,NaN,NaN,13.8,6.7,54.5,NaN,3.8,NaN,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,01,2023,0.757,7318.0,NaN,6.0,7.924,NaN,0.307,1.575,0.196,NaN,3.0,15.296,NaN,11755.0,0.050,0.534,1.441,6.0,4.164,0.584,62709.0,2.997,0.088,2.0,22.618,0.015,54159.0,0.292,91.0,1.0,2.523,0.396,0.020,1.955,NaN,35.0,0.0,0.373,NaN,NaN,11.0,9.4,41.2,NaN,1.7,NaN,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,01,2023,NaN,268.0,NaN,0.0,0.165,NaN,NaN,NaN,NaN,NaN,2.0,0.774,NaN,1417.0,NaN,NaN,NaN,0.0,0.132,NaN,5484.0,NaN,0.466,1.0,2.148,0.023,5413.0,2.296,6.0,0.0,0.152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.9,25.8,NaN,NaN,NaN,NaN,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,01,2023,NaN,14.0,NaN,0.0,0.008,NaN,NaN,NaN,NaN,NaN,0.0,0

In [27]:
tdf = df[df['Facility ID'] == '010001']
tdf['marker'] = tdf['Facility ID'] + ' | ' + tdf['Facility Name']  + ' | ' + tdf['file_month'] + ' | ' + tdf['file_year']
print(tdf.shape)
column = tdf.pop('marker')
tdf.insert(0, column.name, column)
tdf.drop_duplicates(inplace=True)
print(tdf.shape)

print(' ', len(tdf['marker']))
print(' ', len(tdf['marker'].unique()))
markers = tdf['marker'].tolist()
u_markers = tdf['marker'].unique().tolist()
i = str()
for i in u_markers:
    if markers.count(i) > 1:
        print(i)
        break

tdf = tdf[tdf['marker'] == i]
tdf.head(10)

(42, 52)
(42, 52)
  42
  41
010001 | SOUTHEAST HEALTH MEDICAL CENTER | 07 | 2022


,marker,Facility ID,Facility Name,file_month,file_year,Healthcare Associated Infections: CLABSI (SIR),Healthcare Associated Infections: CLABSI Number of Device Days,Healthcare Associated Infections: CLABSI Number of Procedures,Healthcare Associated Infections: CLABSI Observed Cases,Healthcare Associated Infections: CLABSI Predicted Cases,Healthcare Associated Infections: CLABSI SIR (ICUs only),Healthcare Associated Infections: CLABSI lower CL,Healthcare Associated Infections: CLABSI upper CL,Healthcare Associated Infections: CAUTI (SIR),Healthcare Associated Infections: CAUTI Number of Procedures,Healthcare Associated Infections: CAUTI Observed Cases,Healthcare Associated Infections: CAUTI Predicted Cases,Healthcare Associated Infections: CAUTI SIR (ICUs only),Healthcare Associated Infections: CAUTI Urinary Catheter Days,Healthcare Associated Infections: CAUTI lower CL,Healthcare Associated Infections: CAUTI upper CL,Healthcare Associated Infections: MRSA (SIR),Healthcare Associated Infections: MRSA Observed Cases,Healthcare Associated Infections: MRSA Predicted Cases,Healthcare Associated Infections: MRSA lower CL,Healthcare Associated Infections: MRSA patient days,Healthcare Associated Infections: MRSA upper CL,Healthcare Associated Infections: CDIFF (SIR),Healthcare Associated Infections: CDIFF Observed Cases,Healthcare Associated Infections: CDIFF Predicted Cases,Healthcare Associated Infections: CDIFF lower CL,Healthcare Associated Infections: CDIFF patient days,Healthcare Associated Infections: CDIFF upper CL,Healthcare Associated Infections: SSI Colon Number of Procedures,Healthcare Associated Infections: SSI Colon Observed Cases,Healthcare Associated Infections: SSI Colon Predicted Cases,Healthcare Associated Infections: SSI Colon Surgery (SIR),Healthcare Associated Infections: SSI Colon lower CL,Healthcare Associated Infections: SSI Colon upper CL,Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR),Healthcare Associated Infections: SSI Abdominal Number of Procedures,Healthcare Associated Infections: SSI Abdominal Observed Cases,Healthcare Associated Infections: SSI Abdominal Predicted Cases,Healthcare Associated Infections: SSI Abdominal lower CL,Healthcare Associated Infections: SSI Abdominal upper CL,HVBP Total Performance: Abdomen CT Use of Contrast Material,HVBP Total Performance: Breast Cancer Screening Recall Rates,HVBP Total Performance: MRI Lumbar Spine for Low Back Pain,HVBP Total Performance: Mammography Follow-up Rates,HVBP Total Performance: Outpatients who got cardiac imaging stress tests before low-risk outpatient surgery,HVBP Total Performance: Outpatients with brain CT scans who got a sinus CT scan at the same time,HVBP Total Performance: Thorax CT Use of Contrast Material
29065,010001 | SOUTHEAST HEALTH MEDICAL CENTER | 07 ...,010001,SOUTHEAST HEALTH MEDICAL CENTER,07,2022,0.29,9772.0,NaN,3.0,10.332,NaN,0.074,0.79,0.275,NaN,7.0,25.491,NaN,17182.0,0.12,0.543,0.975,8.0,8.208,0.453,101939.0,1.851,0.583,42.0,71.985,0.426,101939.0,0.781,138.0,2.0,4.257,0.47,0.079,1.552,0.0,258.0,0.0,2.412,NaN,1.242,4.0,NaN,NaN,NaN,NaN,NaN,NaN
29066,010001 | SOUTHEAST HEALTH MEDICAL CENTER | 07 ...,010001,SOUTHEAST HEALTH MEDICAL CENTER,07,2022,0.29,9772.0,NaN,3.0,10.332,NaN,0.074,0.79,0.275,NaN,7.0,25.491,NaN,17182.0,0.12,0.543,0.975,8.0,8.208,0.453,101939.0,1.851,0.583,42.0,71.985,0.426,101939.0,0.781,138.0,2.0,4.257,0.47,0.079,1.552,0.0,258.0,0.0,2.412,NaN,1.242,NaN,5.5,42.5,NaN,6.8,NaN,NaN


## Load general hospital info file from hospitals archives

In [ ]:
hosp_df = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/hospital_info_df.pkl.gz', compression='gzip')
hosp_df = hosp_df.filter(items=['Facility ID', 'Hospital Ownership', 'Hospital Type', 'file_month', 'file_year'])
print(hosp_df.shape)
hosp_df.drop_duplicates(inplace=True)
print(hosp_df.shape)

d = {'Acute Care Hospitals': 'Acute Care', 
     np.nan: 'Not given',  
     'Critical Access Hospitals': 'Critical Access', 
     'Acute Care - Department of Defense': 'Acute Care (DOD)',  
     'ACUTE CARE - VETERANS ADMINISTRATION': 'Acute Care (VA)', 
     'Acute Care - Veterans Administration': 'Acute Care (VA)'}
hosp_df['Hospital Type'].replace(to_replace=d, inplace=True)
print(hosp_df['Hospital Type'].unique().tolist(), '\n')

d = {'Veterans Health Administration': 'VA', 
     'Government Federal': 'Government - Federal', 
     'Government-State': 'Government - State', 
     'Voluntary non-profit-Other': 'Voluntary non-profit - Other', 
     'Voluntary non-profit-Private': 'Voluntary non-profit - Private', 
     'Government-Federal': 'Government - Federal'}
hosp_df['Hospital Ownership'].replace(to_replace=d, inplace=True)
print(hosp_df['Hospital Ownership'].unique().tolist(), '\n')

print(hosp_df.shape)
hosp_df = hosp_df[hosp_df['Facility ID'].isin(df['Facility ID'].unique())]
print(hosp_df.shape)

print(len(hosp_df['Facility ID'].unique()), 'unique hospital IDs')     
hosp_df.sort_values(by=['Facility ID', 'file_month', 'file_year'], ascending=False, inplace=True)

print(hosp_df.shape)
hosp_df.drop_duplicates(subset=['Facility ID'], keep='first', inplace=True)
hosp_df.drop(labels=['file_month', 'file_year'], axis=1, inplace=True)
print(hosp_df.shape)
hosp_df.head()

## Load file from HCRIS project that contains Lat's, Lon's, and numbers of beds

In [ ]:
tdf2 = pd.read_pickle(main_dir + 'Files_For_App/GenDat4App.pkl')
tdf2.reset_index(drop=True, inplace=True)
tdf2.columns = tdf2.columns.droplevel([0,1,2])
tdf2.rename(columns={'Type of Control of Hospital (See Table I) (S2_1_C1_21)': 'Hospital Ownership',
                     'Hospital type (modified)': 'Hospital Type',
                     'Hospital Provider Number (PRVDR_NUM)': 'Facility ID',
                     'Hospital State (S2_1_C2_2)': 'State',
                     'Total Facility (S3_1_C2_27)': 'Beds',
                    }, inplace=True)

tdf2 = tdf2.filter(items=['Facility ID', 'Lat', 'Lon', 'Beds', 'State'], axis=1)
tdf2['Beds'] = tdf2.groupby(['Facility ID'])['Beds'].transform('mean')
tdf2['Beds'] = np.round(tdf2['Beds'], 0)

print(tdf2.shape)
tdf2.drop_duplicates(inplace=True)
print(tdf2.shape)
hosp_ids = df['Facility ID'].unique()
tdf2 = tdf2[tdf2['Facility ID'].isin(hosp_ids)]
print(tdf2.shape)
tdf2.head()

## Merge dataframes

In [ ]:
tdf = df.copy(deep=True)

print(tdf.shape)
tdf = tdf.merge(hosp_df, how='outer', on=['Facility ID'])
print(tdf.shape)
tdf = tdf.merge(tdf2, how='outer', on=['Facility ID'])
print(tdf.shape)
tdf.drop_duplicates(inplace=True)
print(tdf.shape)
tdf.head()

## Minor formatting

In [ ]:
cols = ['State', 'Lon', 'Lat', 'Beds', 'Hospital Ownership', 'Hospital Type', 'file_year', 'file_month', 'Facility Name', 'Facility ID']
for c in cols:
    col = tdf.pop(c)
    tdf.insert(0, col.name, col)

labs = list(tdf)
for c in cols:
    labs.remove(c)
    
print(tdf.shape)
tdf = tdf.dropna(subset=labs, how='all')
print(tdf.shape)
tdf.drop_duplicates(subset=cols, inplace=True)
print(tdf.shape)

tdf.head()

## Write file to be used for initiating app

In [ ]:
cols = ['Facility ID', 'Facility Name', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
tdf2 = tdf.filter(items=cols, axis=1)

print(tdf2.shape)
tdf2.drop_duplicates(inplace=True)
print(tdf2.shape)

tdf2.to_csv(main_dir + '/files_for_app/GenDat4App.csv')
tdf2.head()

## Write hospital-specific files

In [ ]:
hospitals = sorted(df['Facility ID'].unique())

for h in hospitals:
    tdf = df[df['Facility ID'] == h]
    df = df[df['Facility ID'] != h]
    
    tdf.sort_values(by=['file_year', 'file_month'], ascending=True, inplace=True)
    tdf.reset_index(drop=True, inplace=True)
    tdf.to_csv(main_dir + '/hospital_files/' + h + '.csv')

## Load and Merge Measurement Dates files

In [ ]:
ls2 = ['HVBP',
       'Outpatient Imaging Efficiency',
       'Payment and Value of Care',
       'Complications and Deaths',
       'Timely and Effective Care',
       'Hospital Readmissions Reduction Program',
       'Hospital Acquired Conditions Reduction Program',
       'HCAHPS',
       'Unplanned Visits',
]

In [ ]:
df = pd.read_csv(main_dir + 'measure_dates/hvbp_measure_dates.csv')
print('df.shape:', df.shape)
df['Measure Name'] = ls2[0] + ': ' + df['Measure Name']


ls1 = ['hvbp_measure_dates_df.csv',
       'Outpatient_Imaging_Efficiency_df.csv', 
       'Payment_and_ValueOfCare_df.csv', 
       'complication_and_deaths_df.csv', 
       'Timely_and_Effective_Care_df.csv',
       'hrrp_df.csv',
       'hacrp_df.csv',
       'hcahps_df.csv',
       'unplanned_visits_df.csv',
      ]

brk = 0
for i, l in enumerate(ls1):
    if i == 0:
        continue
        
    print(l)
    tdf = pd.read_csv(main_dir + 'measure_dates/' + l)
    tdf['Measure Name'] = ls2[i] + ': ' + tdf['Measure Name']
    
    df = pd.concat([df, tdf])
    print('df.shape:', df.shape)
    
    for i in list(df):
        if '_x' in i:
            print('redundant column name:', i)
            brk = 1
            break
            
    if brk == 1: break
    del tdf

df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
print('df.shape:', df.shape)
df.head()


In [ ]:
df.to_csv('~/GitHub/hospitals-data-archive/measure_dates/all_measure_dates.csv')
